In [11]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn
%pip install requests

Looking in indexes: https://download.pytorch.org/whl/cu128
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   -------------------------------- ------- 4/5 [requests]
   ---------------------------------------- 5/5 [requests]

Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch

from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

from torchvision.models import resnet50, ResNet50_Weights
from torchvision import datasets, transforms

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

class_names = ['Aster', 'Daisy', 'Iris', 'Lavender', 'Lily', 'Marigold', 'Orchid', 'Poppy', 'Rose', 'Sunflower']

In [10]:
train_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
])

val_test_transforms = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder('./Dataset/Training Data', transform=train_transforms)
val_dataset   = datasets.ImageFolder('./Dataset/Validation Data', transform=val_test_transforms)
test_dataset  = datasets.ImageFolder('./Dataset/Testing Data', transform=val_test_transforms)

print("Num Dataset: ", str((len(train_dataset) + len(val_dataset) + len(test_dataset))))
print("Num Treino: ", str(len(train_dataset)))
print("Num Validação: ", str(len(val_dataset)))
print("Num Teste: ", str(len(test_dataset)))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Num Dataset:  22355
Num Treino:  15000
Num Validação:  2355
Num Teste:  5000


In [ ]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_features = model.fc.in_features

model.fc = torch.nn.Linear(num_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Training complete! 🎉")

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

In [ ]:
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}\n")

print(classification_report(all_labels, all_preds, target_names=class_names))

In [ ]:
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


In [ ]:
torch.save(model, '/content/drive/MyDrive/Aprendizado de Maquina/Data-ML/resnet50_flowers_dataset.h5')